In [1]:
import pandas as pd
"""
CustumVQE: A custom implementation of the Variational Quantum Eigensolver (VQE) for portfolio optimization.

Classes:
    CustumVQE: A class to perform VQE for portfolio optimization using quantum computing.

Methods:
    __init__(self, gamma=1, B=2, P=1.0, p=2, stepsize=0.02, max_steps=600):
        Initializes the CustumVQE class with quantum and optimization parameters.
    
    _fetch_stock_data(self):
        Fetches stock data and computes the covariance matrix (Sigma) and mean returns (R).
    
    _build_hamiltonian(self):
        Constructs the problem Hamiltonian for the VQE.
    
    ansatz(self, params):
        Defines the ansatz (trial wavefunction) for the quantum circuit.
    
    cost(self, params):
        Defines the cost function for the VQE.
    
    optimize(self):
        Runs the optimizer to minimize the cost function.
    
    probability_circuit(self, params):
        Analyzes the optimized circuit by returning the probabilities of different bit strings.
    
    print_bitstring_probabilities(self, params):
        Prints the probabilities of different bit strings after optimization.
    
    print_all_probabilities(self, params):
        Prints all bit string probabilities and identifies the final solution.
    
    brute_force_solution(self):
        Finds the exact solution by brute force for comparison.
    
    get_probabilities_and_bitstrings(self, params):
        Returns the probabilities and corresponding bit strings for a given set of parameters.
    
    run(self):
        Executes the VQE process, including optimization and brute force solution finding.
"""
import yfinance as yf
import pennylane as qml
from pennylane import numpy as np
import itertools

class CustumVQE:
    def __init__(self, gamma=1, B=2, P=1.0, p=2, stepsize=0.02, max_steps=600):

        # Quantum and optimization parameters
        self.N = 5
        self.gamma = gamma
        self.B = B
        self.P = P
        self.p = p
        self.stepsize = stepsize
        self.max_steps = max_steps

        # Quantum device
        self.dev = qml.device("default.qubit", wires=self.N)

        # Initialize Hamiltonian and Ansatz parameters
        self._fetch_stock_data()

        self.init_params = np.random.rand((p+1) * self.N)

        # QNode for cost function
        self.cost_qnode = qml.QNode(
            self.cost, self.dev, diff_method="parameter-shift")

    def _fetch_stock_data(self):
        """Fetch stock data and compute covariance matrix (Sigma) and mean returns (R)."""
        assets = "AAPL MSFT AMZN TSLA GOOG"
        StockStartDate = '2018-01-01'
        StockEndDate = '2018-12-31'
        interval = '1d'
        df = yf.download(assets, start=StockStartDate,
                         end=StockEndDate, interval=interval)['Adj Close']
        ret = df.pct_change().dropna()
        self.R = ret.mean() * 252
        self.Sigma = ret.cov() * 252

    def _build_hamiltonian(self):
        """Construct the problem Hamiltonian."""
        ZZ = [qml.PauliZ(i) @ qml.PauliZ(j) for i in range(self.N)
              for j in range(i+1, self.N)]
        ZZ_coeff = [0.5 * (self.gamma * self.Sigma.values[i][j] + self.P)
                    for i in range(self.N) for j in range(i+1, self.N)]

        Z = [qml.PauliZ(i) for i in range(self.N)]
        Z_coeff = [
            -0.5 * self.gamma *
            (sum(self.Sigma.values[i][:])) + 0.5 *
            self.R[i] - 0.5 * self.P * (self.N - 2 * self.B)
            for i in range(self.N)
        ]

        C = 0.25 * self.gamma * (sum(sum(self.Sigma.values)) + np.trace(self.Sigma)) - 0.5 * sum(self.R) + \
            0.25 * self.P * (self.N + (self.N - 2 * self.B) ** 2)

        obs = ZZ + Z
        coeffs = ZZ_coeff + Z_coeff
        self.H = qml.Hamiltonian(coeffs, obs, grouping_type="qwc")

    def ansatz(self, params):
        """Enhance performance with using Hardware Efficient Ansatz for circuit."""
        for q in range(self.N):
            qml.RY(params[q], wires=q)
        for d in range(1, self.p + 1):
            for q in range(self.N - 1):
                qml.CNOT(wires=[q, q + 1])
            for q in range(self.N):
                qml.RY(params[d * self.N + q], wires=q)

    def cost(self, params):
        """Define the cost function."""
        self.ansatz(params)
        return qml.expval(self.H)

    def optimize(self):
        self._build_hamiltonian()
        """Run the optimizer to minimize the cost function."""
        opt = qml.QNGOptimizer(stepsize=self.stepsize)
        old_cost = 9999.999999
        params = self.init_params

        for i in range(self.max_steps):
            params = opt.step(self.cost_qnode, params)
            obj_value = self.cost_qnode(params)

            if (i + 1) % 5 == 0:
                print("Cost after step {:5d}: {: .7f}".format(
                    i + 1, obj_value))
                if np.round(old_cost, 7) == np.round(obj_value, 7):
                    break
                else:
                    old_cost = obj_value

        print("Optimized parameters: {}".format(params))
        print("Optimized objective function value: {}".format(obj_value))

    def probability_circuit(self, params):
        """For analyzing the optimized circuit."""
        @qml.qnode(self.dev)
        def circuit():
            self.ansatz(params)
            return qml.probs(wires=range(self.N))
        return circuit()

    def print_bitstring_probabilities(self, params):
        probabilities = self.probability_circuit(params)
        bit_strings = [format(i, '0{}b'.format(self.N))
                       for i in range(2 ** self.N)]
        print("Bit string probabilities:")
        for bit_string, probability in zip(bit_strings, probabilities):
            print(f"{bit_string}: {probability:.4f}")

    def print_all_probabilities(self, params):
        probs = self.probability_circuit(params)
        N = int(np.log2(len(probs)))
        max_index = np.argmax(probs)
        max_prob = probs[max_index]
        print("Final solution: {:0{}b}, with prob={:.5f}".format(
            max_index, N, max_prob))
        for i, prob in enumerate(probs):
            print(
                "Bit string: {:0{}b}, Probability: {:.5f}".format(i, N, prob))

    def brute_force_solution(self):
        """Find the exact solution by brute force."""
        all_combinations = itertools.product([0, 1], repeat=self.N)
        # Set predefined threshold for eigen value comparison
        E_g = 99999.99999
        for x in all_combinations:
            # print(x)

            E = self.gamma * np.dot(x, np.dot(self.Sigma, x)) - \
                np.dot(self.R, x) + self.P * (sum(x) - self.B) ** 2
            print(f"Option Bit String: {x} with Eigeien Value = {E:.4f}")
            if E < E_g:
                E_g = E
                sol = x
                print(f"New optimal found: {sol} with E_g = {E_g:.4f}")
        print("Exact solution:{}".format(sol))
        return sol

    def get_probabilities_and_bitstrings(self, params):
        probabilities = self.probability_circuit(params)
        # Assuming probabilities are for a complete set of basis states
        N = int(np.log2(len(probabilities)))
        bit_strings = [format(i, '0{}b'.format(N)) for i in range(2**N)]
        return probabilities, bit_strings

    def run(self):
        """Run the VQE process."""
        print("Starting optimization...")
        self.optimize()
        print("Finding the exact solution using brute force...")
        self.brute_force_solution()
        # self.validate_quantum_soluti
        # on(self.init_params)

In [2]:
vqe = CustumVQE()

[*********************100%%**********************]  5 of 5 completed


In [3]:
vqe.run()

Starting optimization...
Cost after step     5: -0.5623631
Cost after step    10: -0.8374151
Cost after step    15: -0.9445573
Cost after step    20: -1.0038574
Cost after step    25: -1.0493703
Cost after step    30: -1.0913070
Cost after step    35: -1.1331054
Cost after step    40: -1.1753896
Cost after step    45: -1.2175754
Cost after step    50: -1.2589504
Cost after step    55: -1.2992212
Cost after step    60: -1.3384141
Cost after step    65: -1.3764071
Cost after step    70: -1.4125324
Cost after step    75: -1.4455177
Cost after step    80: -1.4738293
Cost after step    85: -1.4963079
Cost after step    90: -1.5127405
Cost after step    95: -1.5239300
Cost after step   100: -1.5312290
Cost after step   105: -1.5359741
Cost after step   110: -1.5391805
Cost after step   115: -1.5415087
Cost after step   120: -1.5433519
Cost after step   125: -1.5449347
Cost after step   130: -1.5463855
Cost after step   135: -1.5477796
Cost after step   140: -1.5491636
Cost after step   145: 